In [5]:
import time
import pygame
import numpy as np
import random


COLOR_BG = (10, 10, 10) 
COLOR_GRID = (40, 40, 40) 
COLOR_DIE_NEXT = (170, 170, 170) 
COLOR_ALIVE_NEXT = (255, 182, 193)
COLOR_RANDOM = (255, 182, 193)

def update(screen, cells, size, with_progress=False):
    update_cells = np.zeros(cells.shape, dtype=int)

    rows, cols = cells.shape
    for row, col in np.ndindex(cells.shape):
        alive = sum(
            cells[(row + i) % rows, (col + j) % cols]
            for i in [-1, 0, 1]
            for j in [-1, 0, 1]
            if not (i == 0 and j == 0)
        )

        color = COLOR_BG if cells[row, col] == 0 else COLOR_ALIVE_NEXT

        if cells[row, col] == 1:
            if alive < 2 or alive > 3:
                if with_progress:
                    color = COLOR_DIE_NEXT
            elif 2 <= alive <= 3:
                update_cells[row, col] = 1
                if with_progress:
                    color = COLOR_ALIVE_NEXT
        else:
            if alive == 3:
                update_cells[row, col] = 1
                if with_progress:
                    color = COLOR_ALIVE_NEXT

        pygame.draw.rect(screen, color, (col * size, row * size, size - 1, size - 1))

    return update_cells

def reset(cells):
    cells[:, :] = 0
    return cells

def generate_random(cells):
    cells[:, :] = np.random.randint(0, 2, size=cells.shape)
    return cells

def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    clock = pygame.time.Clock()

    cells = np.zeros((60, 80))  
    size = 10  

    screen.fill(COLOR_GRID)
    update(screen, cells, size)
    pygame.display.flip()
    pygame.display.update()

    running = False
    drawing = False

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    running = not running
                    update(screen, cells, size)
                    pygame.display.update()
                if event.key == pygame.K_q: 
                    cells = reset(cells)
                    update(screen, cells, size)
                    pygame.display.update()
                if event.key == pygame.K_g:  
                    cells = generate_random(cells)
                    update(screen, cells, size)
                    pygame.display.update()
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                drawing = True
            elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
                drawing = False
            elif event.type == pygame.MOUSEMOTION and drawing:
                pos = pygame.mouse.get_pos()
                # Ensure the indices do not go out of bounds
                cells[min(pos[1] // size, cells.shape[0] - 1), min(pos[0] // size, cells.shape[1] - 1)] = 1
                update(screen, cells, size)
                pygame.display.update()

        screen.fill(COLOR_GRID)

        if running:
            cells = update(screen, cells, size, with_progress=True)
            pygame.display.update()

        clock.tick(10)  
        time.sleep(0.001)

if __name__ == '__main__':
    main()